In [1]:
import os
from contextlib import redirect_stdout

import sys
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Init

In [2]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

var_df = add_water_year_column(df)

df.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else col for col in df.columns]


C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:174: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


## Define variables

In [3]:
hist_percentiles=[0.2, 0.333, 0.5] # percentiles for historical record
tier_thresholds=(0.9, 0.67, 0.3) # tier assignment thresholds
cdec_calsim_vars = {
    "SHA": "S_SHSTA",
    "CLE": "S_TRNTY",
    "ORO": "S_OROVL",
    "FOL": "S_FOLSM",
    "NML": "S_MELON",
    "MIL": "S_MLRTN",
    "LUS": "S_SLUIS_SWP",
    "SLF": "S_SLUIS_CVP"
} # mapping between cdec and calsim var names

cdec_file = "CDEC_Station_Key.csv" # cdec key file name
metadata_file = "reservoir_metadata_table.csv" # reservoir metadata output file name

variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MLRTN_", "S_MELON_"]
variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL4DV", "S_SLUIS_CVPLEVEL4DV", "S_SLUIS_SWPLEVEL4DV", "S_MLRTNLEVEL4", "S_MELONLEVEL4DV"]
variables_capacity = ["S_SHSTALEVEL6DV", "S_OROVLLEVEL6DV", "S_FOLSMLEVEL6DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV", "S_MLRTNLEVEL5DV", "S_MELONLEVEL5DV"]
units = "TAF"

## Create Directories and define filenames

In [4]:
def find_repo_root(marker_dirs=("CalSim3_Model_Runs", "coeqwal")) -> Path:
    """
    Walk upward from the script (or notebook CWD) until we find a directory
    that contains the expected project folders (e.g., 'CalSim3_Model_Runs' and 'coeqwal').
    """
    # 1) Optional env override
    env_root = os.environ.get("DSP_REPO_ROOT")
    if env_root:
        root = Path(env_root).expanduser().resolve()
        if all((root / m).exists() for m in marker_dirs):
            return root

    # 2) Start from script dir if available; else notebook working dir
    try:
        start = Path(__file__).resolve().parent
    except NameError:
        start = Path.cwd().resolve()

    # 3) Walk up until markers are found
    for parent in [start] + list(start.parents):
        if all((parent / m).exists() for m in marker_dirs):
            return parent

    raise FileNotFoundError(
        "Could not locate repo root containing: "
        + ", ".join(marker_dirs)
        + ". Set DSP_REPO_ROOT env var or run from inside the repo."
    )

repo_root = find_repo_root()
output_dir = repo_root / "CalSim3_Model_Runs" / "Scenarios" / "Performance_Metrics"
data_output_dir = output_dir / "Metrics" / "Reservoir_Storage"
data_output_dir.mkdir(parents=True, exist_ok=True)

Apr_Storage_Volume_file = "April_Reservoir_Storage_Volume.csv"
Sep_Storage_Volume_file = "September_Reservoir_Storage_Volume.csv"
Apr_Storage_Volume_CV_file = "April_Reservoir_Storage_Volume_CV.csv"
Sep_Storage_Volume_CV_file = "September_Reservoir_Storage_Volume_CV.csv"
Apr_Storage_Percent_file = "April_Reservoir_Storage_Percent.csv"
Sep_Storage_Percent_file = "September_Reservoir_Storage_Percent.csv"
Apr_Storage_Percent_CV_file = "April_Reservoir_Storage_Percent_CV.csv"
Sep_Storage_Percent_CV_file = "September_Reservoir_Storage_Percent_CV.csv"
Spill_Frequency_file = "Reservoir_Spill_Frequency.csv"
Spill_Probability_file = "Reservoir_Spill_Probability.csv"

Apr_Storage_Volume_path = data_output_dir / Apr_Storage_Volume_file
Sep_Storage_Volume_path = data_output_dir / Sep_Storage_Volume_file
Apr_Storage_Volume_CV_path = data_output_dir / Apr_Storage_Volume_CV_file
Sep_Storage_Volume_CV_path = data_output_dir / Sep_Storage_Volume_CV_file
Apr_Storage_Percent_path = data_output_dir / Apr_Storage_Percent_file
Sep_Storage_Percent_path = data_output_dir / Sep_Storage_Percent_file
Apr_Storage_Percent_CV_path = data_output_dir / Apr_Storage_Percent_CV_file
Sep_Storage_Percent_CV_path = data_output_dir / Sep_Storage_Percent_CV_file
Spill_Frequency_path = data_output_dir / Spill_Frequency_file
Spill_Probability_path = data_output_dir / Spill_Probability_file


## Aux functions

In [5]:
import pandas as pd
import re

def find_calsim_model_root(start_dir=os.getcwd(), folder_name="CalSim3_Model_Runs"):
    current = start_dir
    while True:
        if folder_name in os.listdir(current):
            return os.path.join(current, folder_name)
        parent = os.path.dirname(current)
        if parent == current:
            raise FileNotFoundError(f"Could not find {folder_name} in any parent directories.")
        current = parent

def get_relative_folder(full_folder_path, known_tail):

    full_folder_path = os.path.normpath(full_folder_path)
    known_tail = os.path.normpath(known_tail)
    
    # Find where the known folder structure starts
    idx = full_folder_path.lower().find(known_tail.lower())
    
    if idx == -1:
        raise ValueError("Known tail not found in full path.")
    
    relative_suffix = full_folder_path[idx:]
    
    return os.path.join("..", "..", relative_suffix)

def set_column_level_to_value(df, level, value):
    """
    Set all entries in a specified MultiIndex column level to a constant value.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with MultiIndex columns
    level : int or str
        Column level index (e.g., 5) or level name
    value : str
        Value to assign to the entire level

    Returns
    -------
    pd.DataFrame
        Copy of df with updated column level
    """
    df = df.copy()

    if not isinstance(df.columns, pd.MultiIndex):
        raise TypeError("DataFrame columns must be a MultiIndex")

    lvl = df.columns._get_level_number(level)

    # Replace values at the selected level
    new_tuples = [
        tuple(value if i == lvl else v for i, v in enumerate(col))
        for col in df.columns
    ]

    df.columns = pd.MultiIndex.from_tuples(
        new_tuples, names=df.columns.names
    )

    return df

def compute_percent_of_capacity(
    storage_df,
    capacity_df,
    var_level=1,      # level containing S_FOLSM_s0001
    type_level=5      # PER-AVER → PERCENT
):
    """
    Compute storage as percent of capacity by matching
    (base variable, scenario) pairs.
    """

    if not isinstance(storage_df.columns, pd.MultiIndex):
        raise TypeError("storage_df must have MultiIndex columns")
    if not isinstance(capacity_df.columns, pd.MultiIndex):
        raise TypeError("capacity_df must have MultiIndex columns")

    lvl = storage_df.columns._get_level_number(var_level)

    def split_var_and_scen(s):
        s = str(s)
        base, scen = s.rsplit("_s", 1)
        return base, f"s{scen}"

    # ---- Build lookups ----
    def build_lookup(df):
        lookup = {}
        for col in df.columns:
            base, scen = split_var_and_scen(col[lvl])
            lookup[(base, scen)] = col
        return lookup

    sto_lookup = build_lookup(storage_df)
    cap_lookup = build_lookup(capacity_df)

    common_keys = sto_lookup.keys() & cap_lookup.keys()
    if not common_keys:
        raise ValueError("No matching (variable, scenario) pairs found.")

    # ---- Build aligned frames ----
    sto_cols = []
    cap_cols = []

    for key in sorted(common_keys):
        sto_cols.append(sto_lookup[key])
        cap_cols.append(cap_lookup[key])

    sto = storage_df.loc[:, sto_cols]
    cap = capacity_df.loc[:, cap_cols]

    # Force identical column labels for arithmetic
    cap.columns = sto.columns

    pct_df = sto.divide(cap).multiply(100)

    # Set type level to PERCENT
    pct_df = set_column_level_to_value(
        pct_df,
        level=type_level,
        value="PERCENT"
    )

    return pct_df

def compute_annual_means_list(
    df,
    vars,
    units="TAF",
    months=None,
):
    annual_means = []

    for var in vars:
        subset_df = create_subset_unit(df, var, units)

        subset_df = add_water_year_column(subset_df)

        if months is not None:
            subset_df = subset_df[subset_df.index.month.isin(months)]

        ann_mean = subset_df.groupby("WaterYear").mean()

        annual_means.append(ann_mean)

    # result = pd.concat(
    #     annual_means,
    #     axis=1,
    #     keys=vars,
    #     names=["Variable"]
    # )
    
    result = pd.concat(annual_means, axis=1)

    return result
    

def split_var_scenario(col_name):
    """
    'S_FOLSM_s0001' -> ('S_FOLSM', 's0001')
    """
    m = re.match(r"(.*)_s(\d+)$", col_name)
    if not m:
        return col_name, None
    return m.group(1), f"s{m.group(2)}"

def compute_cv_scenario_variable(df, var_level="B"):
    """
    Returns:
        rows    = scenarios (s0001, s0002, ...)
        columns = base variable names (no _sXXXX)
        values  = coefficient of variation
    """

    records = []

    for col in df.columns:
        var_scen_name = col[df.columns.names.index(var_level)]
        base_var, scenario = split_var_scenario(var_scen_name)

        if scenario is None:
            continue

        series = df[col].dropna()

        mean = series.mean()
        std  = series.std()

        if mean == 0 or series.empty:
            cv = float("nan")
        else:
            cv = std / mean

        records.append({
            "Scenario": scenario,
            "Variable": base_var,
            "CV": cv
        })

    cv_df = (
        pd.DataFrame(records)
        .pivot(index="Scenario", columns="Variable", values="CV")
        .sort_index()
    )

    return cv_df


def strip_leveldv_from_columns(df, level_name="B"):
    """
    Remove 'LEVEL<digits>DV' from a specified MultiIndex column level.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with MultiIndex columns
    level_name : str or int, default "B"
        Column level name or index containing variable names

    Returns
    -------
    pd.DataFrame
        DataFrame with cleaned column names
    """
    if not isinstance(df.columns, pd.MultiIndex):
        raise ValueError("DataFrame columns must be a MultiIndex")

    # Resolve level index
    if isinstance(level_name, str):
        if level_name not in df.columns.names:
            raise ValueError(f"Level '{level_name}' not found in columns")
        lvl = df.columns.names.index(level_name)
    else:
        lvl = level_name

    # Rebuild columns safely
    new_cols = []
    for col in df.columns:
        col = list(col)
        col[lvl] = re.sub(r"\s*LEVEL\d+DV", "", col[lvl])
        col[lvl] = re.sub(r"\s+", "", col[lvl])
        new_cols.append(tuple(col))

    df = df.copy()
    df.columns = pd.MultiIndex.from_tuples(
        new_cols, names=df.columns.names
    )

    return df

def strip_level_from_columns(df, level_name="B"):
    """
    Remove 'LEVEL<digits>' from a specified MultiIndex column level.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with MultiIndex columns
    level_name : str or int, default "B"
        Column level name or index containing variable names

    Returns
    -------
    pd.DataFrame
        DataFrame with cleaned column names
    """
    if not isinstance(df.columns, pd.MultiIndex):
        raise ValueError("DataFrame columns must be a MultiIndex")

    # Resolve level index
    if isinstance(level_name, str):
        if level_name not in df.columns.names:
            raise ValueError(f"Level '{level_name}' not found in columns")
        lvl = df.columns.names.index(level_name)
    else:
        lvl = level_name

    # Rebuild columns safely
    new_cols = []
    for col in df.columns:
        col = list(col)
        col[lvl] = re.sub(r"\s*LEVEL\d", "", col[lvl])
        col[lvl] = re.sub(r"\s+", "", col[lvl])
        new_cols.append(tuple(col))

    df = df.copy()
    df.columns = pd.MultiIndex.from_tuples(
        new_cols, names=df.columns.names
    )

    return df
    
def freq_and_prob_storage_ge_flood(
    storage_df: pd.DataFrame,
    flood_df: pd.DataFrame,
    var_level="B",
):
    """
    Returns two DataFrames:
      1) frequency: count of WaterYears where storage >= flood
      2) probability: frequency / number of WaterYears

    Rows    -> scenario (s0001, s0002, ...)
    Columns -> base variable (S_SHSTA, S_MELON, ...)
    """

    if not isinstance(storage_df.columns, pd.MultiIndex):
        raise TypeError("storage_df must have MultiIndex columns")
    if not isinstance(flood_df.columns, pd.MultiIndex):
        raise TypeError("flood_df must have MultiIndex columns")

    lvl = storage_df.columns._get_level_number(var_level)
    n_years = storage_df.shape[0]
    n_months = n_years * 12

    def split_base_and_scen(s):
        m = re.match(r"(.*)_s(\d+)$", str(s))
        if m is None:
            return None, None
        return m.group(1), f"s{m.group(2)}"

    def build_lookup(df):
        lookup = {}
        for col in df.columns:
            base, scen = split_base_and_scen(col[lvl])
            if base is not None:
                lookup[(base, scen)] = col
        return lookup

    sto_lookup = build_lookup(storage_df)
    flo_lookup = build_lookup(flood_df)

    common_keys = sto_lookup.keys() & flo_lookup.keys()
    if not common_keys:
        raise ValueError("No matching (variable, scenario) pairs found.")

    records = []

    for base, scen in sorted(common_keys):
        sto_col = sto_lookup[(base, scen)]
        flo_col = flo_lookup[(base, scen)]

        mask = storage_df[sto_col] >= flood_df[flo_col]

        freq = int(mask.sum())
        prob = freq / n_months
        records.append(
            {
                "scenario": scen,
                "variable": base,
                "frequency": freq,
                "probability": prob,
            }
        )

    freq_df = (
        pd.DataFrame(records)
        .pivot(index="scenario", columns="variable", values="frequency")
        .sort_index()
        .sort_index(axis=1)
    )

    prob_df = (
        pd.DataFrame(records)
        .pivot(index="scenario", columns="variable", values="probability")
        .sort_index()
        .sort_index(axis=1)
    )

    freq_df = freq_df.astype("Int64")

    return freq_df, prob_df

def generate_tier_assignment_matrix(
    df, cdec_df, start_date="1921-10-01",
    percentiles=[0.25, 0.5, 0.9], tier_thresholds=(0.9, 0.5, 0.2), 
    saveprobs = False, verbose = False
):
    def load_historical_storage_csv(filepath):
        df_raw = pd.read_csv(filepath, header=None)
        start_row = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains('RESERVOIR STORAGE').any(), axis=1)].index[0]
        df_data = pd.read_csv(filepath, skiprows=start_row)
        df_data.columns = df_data.columns.str.strip()
        df_data["DATE"] = pd.to_datetime(df_data.iloc[:, 0], format="%Y-%m-%d", errors="coerce")
        df_data = df_data.dropna(subset=["DATE"])
        storage_col = next((col for col in df_data.columns if "RESERVOIR STORAGE" in col.upper()), None)
        df_data["STORAGE"] = pd.to_numeric(df_data[storage_col], errors="coerce")
        df_data = df_data.dropna(subset=["STORAGE"])
        return df_data[["DATE", "STORAGE"]]

    def extract_historical_thresholds(df, percentiles):
        may = df[df["DATE"].dt.month == 5]
        may_1 = may.groupby(may["DATE"].dt.year).first()
        thresholds = may_1["STORAGE"].quantile(percentiles)
        return thresholds / 1000  # Convert AF to TAF

    def extract_variable_by_scenario(df, variable):
        return df[
            [col for col in df.columns
             if variable in col and "_STORAGE_" in col and "LEVEL" not in col.upper()]
        ]

    def assign_tiers_from_calsim(var_df, thresholds, date_series, var, tier_thresholds, saveprobs = saveprobs, verbose = verbose):
        tier_rows = []

        for col in var_df.columns:
            match = re.search(r's\d{4}', col)
            if not match:
                continue
            sid = match.group(0)

            series = var_df[col].copy()
            if not pd.api.types.is_datetime64_any_dtype(series.index):
                series.index = date_series

            april_series = series[series.index.month == 4]
            april_by_year = april_series.groupby(april_series.index.year).last()

            if verbose:
                print(f"\n Scenario {sid} ({var})")
                print("  April-end values:")
                print(april_by_year.head())

            if april_by_year.empty:
                print(f" No April data found for {var} in scenario {sid}")
                continue

            low_thresh = thresholds[percentiles[0]]
            mid_thresh = thresholds[percentiles[1]]
            high_thresh = thresholds[percentiles[2]]

            top = (april_by_year >= high_thresh).sum()
            mid = ((april_by_year >= mid_thresh) & (april_by_year < high_thresh)).sum()
            low = ((april_by_year >= low_thresh) & (april_by_year < mid_thresh)).sum()
            bot = (april_by_year < low_thresh).sum()
            total = len(april_by_year)

            top_frac = top / total
            mid_frac = mid / total
            low_frac = low / total
            bot_frac = bot / total

            tt1, tt2, tt3 = tier_thresholds
            if top_frac >= tt1:
                tier = 1
            elif (top_frac + mid_frac) >= tt2:
                tier = 2
            elif (top_frac + mid_frac) >= tt3:
                tier = 3
            else:
                tier = 4

            tier_rows.append({
                "Scenario": sid,
                "Variable": var,
                "TopProb": round(top_frac, 3),
                "MidProb": round(mid_frac, 3),
                "LowProb": round(low_frac, 3),
                "BotProb": round(bot_frac, 3),
                "StorageTier": tier
            })

        return pd.DataFrame(tier_rows).drop_duplicates(subset=["Scenario", "Variable"])

    try:
        base_model_dir = find_calsim_model_root()
    except FileNotFoundError as e:
        print(e)
        return pd.DataFrame()

    hist_data_dir = os.path.join(base_model_dir, "Scenarios", "CDEC_Historical_Monthly_Storage")
    tiers_output_dir = os.path.join(
        base_model_dir, "Scenarios", "Performance_Metrics", "Tiered_Outcome_Measures", "Reservoir_Storage", "Tiers"
    )
    metrics_output_dir = os.path.join(
        base_model_dir, "Scenarios", "Performance_Metrics", "Metrics", "Reservoir_Storage"
    )
    os.makedirs(tiers_output_dir, exist_ok=True)
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df.index = pd.date_range(start=start_date, periods=len(df), freq="MS")
    df["DATE"] = df.index  


    tier_matrix = pd.DataFrame()
    prob_matrix = pd.DataFrame()

    for _, row in cdec_df.iterrows():
        var = row["CalSim_Variable"]
        file = row["filename"]
        label = f"{var}_Storage"

        if verbose:
            print(f"\n Processing reservoir: {row['ReservoirName']}")
            print(f"  ↳ CalSim variable: {var}")
            print(f"  ↳ Historical file: {file}")

        try:
            hist_path = os.path.join(hist_data_dir, file)
            hist_df = load_historical_storage_csv(hist_path)
            thresholds = extract_historical_thresholds(hist_df, percentiles)
            if verbose:
                print(f"  ↳ Historical thresholds: {thresholds.to_dict()}")

            var_df = extract_variable_by_scenario(df, var)
            if verbose:
                print(f"  ↳ Matched CalSim columns: {var_df.columns.tolist()}")

            if var_df.empty:
                print(f" No CalSim data found for variable {var}")
                continue

            tier_df = assign_tiers_from_calsim(var_df, thresholds, df["DATE"], var, tier_thresholds)

            for _, r in tier_df.iterrows():
                sid = r["Scenario"]
                prob_matrix.loc[sid, f"{label}_TopProb"] = r["TopProb"]
                prob_matrix.loc[sid, f"{label}_MidProb"] = r["MidProb"]
                prob_matrix.loc[sid, f"{label}_LowProb"] = r["LowProb"]
                prob_matrix.loc[sid, f"{label}_BotProb"] = r["BotProb"]
                tier_matrix.loc[sid, f"{label}_Tier"] = r["StorageTier"]

        except Exception as e:
            print(f" Failed to process {var}: {e}")
            continue

    tier_matrix.index.name = "Scenario"
    prob_matrix.index.name = "Scenario"

    if verbose:
        print("tier_matrix:")
        print(tier_matrix.head(2))
        print("prob_matrix:")
        print(prob_matrix.head(2))

    # check if output directory exists
    if not os.path.exists(tiers_output_dir):
        # print warning
        print("Warning: directory " + tiers_output_dir + " does not exists and will be created")
        
        # Create the directory
        os.makedirs(tiers_output_dir)

    if not os.path.exists(metrics_output_dir):
        # print warning
        print("Warning: directory " + metrics_output_dir + " does not exists and will be created")
        
        # Create the directory
        os.makedirs(metrics_output_dir)

    tiers_output_path = os.path.join(tiers_output_dir, "tier_assignment.csv")
    metrics_output_path = os.path.join(metrics_output_dir, "level_probabilities.csv")
    tier_matrix.to_csv(tiers_output_path)
    print(f"\n Tier assignment CSV saved to:\n{tiers_output_path}")
    if saveprobs:
        prob_matrix.to_csv(metrics_output_path)
        print(f"\n Level probabilities CSV saved to:\n{metrics_output_path}")

    return tier_matrix, prob_matrix


## CDEC<-> Calsim table

In [6]:
base_model_dir = find_calsim_model_root()

hist_full_subdir = os.path.join(base_model_dir, "Scenarios", "CDEC_Historical_Monthly_Storage")
known_tail = r"CalSim3_Model_Runs\Scenarios\CDEC_Historical_Monthly_Storage"

hist_subdir = get_relative_folder(hist_full_subdir, known_tail)
print("hist_subdir: " + hist_subdir)

key_file = os.path.join(hist_subdir, cdec_file)
cdec_df = pd.read_csv(key_file)
cdec_df.columns = cdec_df.columns.str.strip()

cdec_df["CalSim_Variable"] = cdec_df["CDEC_Key"].map(cdec_calsim_vars)
cdec_df = cdec_df.dropna(subset=["CalSim_Variable"])

file_prefix = "20250506_MonthlyResData_CDEC-"
cdec_df["filename"] = file_prefix + cdec_df["CDEC_Key"] + ".csv"
cdec_df["file_path"] = cdec_df["filename"].apply(lambda f: os.path.join(hist_subdir, f))

final_table = cdec_df[["ReservoirName", "CDEC_Key", "CalSim_Variable", "filename", "file_path"]]
print(final_table)

# check if output directory exists
if not os.path.exists(hist_subdir):
    # print warning
    print("Warning: directory " + hist_subdir + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(hist_subdir)
    
output_path = os.path.join(hist_subdir, metadata_file)
final_table.to_csv(output_path, index=False)
print(f"\n Metadata table saved to: {output_path}")



hist_subdir: ..\..\CalSim3_Model_Runs\Scenarios\CDEC_Historical_Monthly_Storage
        ReservoirName CDEC_Key CalSim_Variable  \
0              Shasta      SHA         S_SHSTA   
1             Trinity      CLE         S_TRNTY   
2            Oroville      ORO         S_OROVL   
3              Folsom      FOL         S_FOLSM   
4         New Melones      NML         S_MELON   
5           Millerton      MIL         S_MLRTN   
6  San Luis - Federal      SLF     S_SLUIS_CVP   
7    San Luis - State      LUS     S_SLUIS_SWP   

                               filename  \
0  20250506_MonthlyResData_CDEC-SHA.csv   
1  20250506_MonthlyResData_CDEC-CLE.csv   
2  20250506_MonthlyResData_CDEC-ORO.csv   
3  20250506_MonthlyResData_CDEC-FOL.csv   
4  20250506_MonthlyResData_CDEC-NML.csv   
5  20250506_MonthlyResData_CDEC-MIL.csv   
6  20250506_MonthlyResData_CDEC-SLF.csv   
7  20250506_MonthlyResData_CDEC-LUS.csv   

                                           file_path  
0  ..\..\CalSim3_Model_Run

## Compute tiers

In [7]:
tier_df, prob_df = generate_tier_assignment_matrix(
    # df, cdec_df, hist_data_dir,
    df, cdec_df, hist_subdir,
    percentiles=hist_percentiles,
    tier_thresholds=tier_thresholds
)



 Tier assignment CSV saved to:
C:\Dino\COEQWAL\COEQWAL_GIT\CalSim3_Model_Runs\Scenarios\Performance_Metrics\Tiered_Outcome_Measures\Reservoir_Storage\Tiers\tier_assignment.csv


In [8]:
tier_df.head(5)

,S_SHSTA_Storage_Tier,S_TRNTY_Storage_Tier,S_OROVL_Storage_Tier,S_FOLSM_Storage_Tier,S_MELON_Storage_Tier,S_MLRTN_Storage_Tier,S_SLUIS_CVP_Storage_Tier,S_SLUIS_SWP_Storage_Tier
Scenario,,,,,,,,
s0001,2.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0
s0002,2.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0
s0003,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0
s0004,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0
s0005,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0


In [9]:
prob_df.head(5)

,S_SHSTA_Storage_TopProb,S_SHSTA_Storage_MidProb,S_SHSTA_Storage_LowProb,S_SHSTA_Storage_BotProb,S_TRNTY_Storage_TopProb,S_TRNTY_Storage_MidProb,S_TRNTY_Storage_LowProb,S_TRNTY_Storage_BotProb,S_OROVL_Storage_TopProb,S_OROVL_Storage_MidProb,...,S_MLRTN_Storage_LowProb,S_MLRTN_Storage_BotProb,S_SLUIS_CVP_Storage_TopProb,S_SLUIS_CVP_Storage_MidProb,S_SLUIS_CVP_Storage_LowProb,S_SLUIS_CVP_Storage_BotProb,S_SLUIS_SWP_Storage_TopProb,S_SLUIS_SWP_Storage_MidProb,S_SLUIS_SWP_Storage_LowProb,S_SLUIS_SWP_Storage_BotProb
Scenario,,,,,,,,,,,,,,,,,,,,,
s0001,0.55,0.25,0.06,0.14,0.42,0.10,0.24,0.24,0.17,0.51,...,0.15,0.53,0.56,0.04,0.22,0.18,0.31,0.09,0.04,0.56
s0002,0.50,0.23,0.11,0.16,0.40,0.09,0.22,0.29,0.15,0.51,...,0.14,0.54,0.52,0.07,0.16,0.25,0.33,0.08,0.08,0.51
s0003,0.39,0.25,0.12,0.24,0.42,0.08,0.19,0.31,0.10,0.43,...,0.07,0.23,0.41,0.11,0.10,0.38,0.32,0.06,0.05,0.57
s0004,0.34,0.23,0.14,0.29,0.38,0.09,0.14,0.39,0.08,0.42,...,0.10,0.21,0.38,0.11,0.09,0.42,0.31,0.08,0.08,0.53
s0005,0.31,0.21,0.15,0.33,0.33,0.13,0.10,0.44,0.08,0.41,...,0.10,0.23,0.37,0.07,0.10,0.46,0.30,0.09,0.07,0.54


## Compute metrics

In [10]:
storage_df = create_subset_list(var_df, variables_storage)
storage_df.head(5)

A                 CALSIM                                            \
B          S_FOLSM_s0001 S_MELON_s0001 S_MLRTN_s0001 S_OROVL_s0001   
C                STORAGE       STORAGE       STORAGE       STORAGE   
D                   1MON          1MON          1MON          1MON   
E                 L2020A        L2020A        L2020A        L2020A   
F               PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units                TAF           TAF           TAF           TAF   
1921-10-31     592.00000     1400.5924     247.27315     2268.9397   
1921-11-30     497.46445     1409.5559     259.80800     2198.0598   
1921-12-31     508.00860     1432.6313     319.01993     2245.1772   
1922-01-31     523.46387     1453.0829     386.45508     2241.3930   
1922-02-28     567.00000     1540.3782     439.00000     2509.7190   

A                                                                           \
B          S_SHSTA_s0001 S_SLUIS_CVP_s0001 S_SLUIS_SWP_s0001 S_TRNTY_s0001   
C                STORAGE           STORAGE           STORAGE       STORAGE   
D                   1MON              1MON              1MON          1MON   
E                 L2020A            L2020A            L2020A        L2020A   
F               PER-AVER          PER-AVER          PER-AVER      PER-AVER   
Units                TAF               TAF               TAF           TAF   
1921-10-31     3057.8250         202.58127         462.50110     1771.0225   
1921-11-30     2908.5920         357.79940         364.16653     1734.6102   
1921-12-31     3076.5317         567.35187         543.17050     1740.1699   
1922-01-31     3204.3298         732.15686         683.54144     1739.6285   
1922-02-28     3585.4524         936.83940         885.41895     1763.0382   

A                                       ...                                  \
B          S_FOLSM_s0002 S_MELON_s0002  ... S_SLUIS_SWP_s0062 S_TRNTY_s0062   
C                STORAGE       STORAGE  ...           STORAGE       STORAGE   
D                   1MON          1MON  ...              1MON          1MON   
E                 L2020A        L2020A  ...            L2020A        L2020A   
F               PER-AVER      PER-AVER  ...          PER-AVER      PER-AVER   
Units                TAF           TAF  ...               TAF           TAF   
1921-10-31     591.41130     1424.3630  ...         462.50110     1764.1914   
1921-11-30     495.81158     1433.3606  ...         339.54898     1722.9004   
1921-12-31     529.82180     1458.4075  ...         287.13590     1732.9128   
1922-01-31     567.00000     1486.7411  ...         432.07388     1753.9695   
1922-02-28     567.00000     1610.4905  ...         708.50400     1818.0074   

A                                                                   \
B          S_FOLSM_s0063 S_MELON_s0063 S_MLRTN_s0063 S_OROVL_s0063   
C                STORAGE       STORAGE       STORAGE       STORAGE   
D                   1MON          1MON          1MON          1MON   
E                 L2020A        L2020A        L2020A        L2020A   
F               PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units                TAF           TAF           TAF           TAF   
1921-10-31     601.37244     1415.1685     245.21529     2277.8574   
1921-11-30     494.55954     1425.5186     254.95457     2253.2383   
1921-12-31     567.00000     1454.5138     316.10970     2444.8508   
1922-01-31     567.00000     1486.7758     408.47296     2500.9300   
1922-02-28     567.00000     1617.1501     439.00000     2712.6000   

A                                                                           
B          S_SHSTA_s0063 S_SLUIS_CVP_s0063 S_SLUIS_SWP_s0063 S_TRNTY_s0063  
C                STORAGE           STORAGE           STORAGE       STORAGE  
D                   1MON              1MON              1MON          1MON  
E                 L2020A            L2020A            L2020A        L2020A  
F               PER-AVER          PER-AVER     

In [11]:
flood_df = create_subset_list(var_df, variables_floodpool)
# flood_df.to_csv("flood_df.csv")
flood_df.head(5)

A                         CALSIM                                            \
B          S_FOLSMLEVEL5DV_s0001 S_MELONLEVEL4DV_s0001 S_MLRTNLEVEL4_s0001   
C                  STORAGE-LEVEL         STORAGE-LEVEL       STORAGE-LEVEL   
D                           1MON                  1MON                1MON   
E                         L2020A                L2020A              L2020A   
F                       PER-AVER              PER-AVER            PER-AVER   
Units                        TAF                   TAF                 TAF   
1921-10-31                 712.0                1970.0               439.0   
1921-11-30                 567.0                1970.0               439.0   
1921-12-31                 567.0                1970.0               439.0   
1922-01-31                 567.0                1970.0               439.0   
1922-02-28                 567.0                1970.0               439.0   

A                                                       \
B          S_OROVLLEVEL5DV_s0001 S_SHSTALEVEL5DV_s0001   
C                  STORAGE-LEVEL         STORAGE-LEVEL   
D                           1MON                  1MON   
E                         L2020A                L2020A   
F                       PER-AVER              PER-AVER   
Units                        TAF                   TAF   
1921-10-31                3057.0              3250.000   
1921-11-30                3057.0              3252.100   
1921-12-31                2828.8              3359.160   
1922-01-31                3000.0              3833.280   
1922-02-28                2712.6              4139.735   

A                                                               \
B          S_SLUIS_CVPLEVEL4DV_s0001 S_SLUIS_SWPLEVEL4DV_s0001   
C                      STORAGE-LEVEL             STORAGE-LEVEL   
D                               1MON                      1MON   
E                             L2020A                    L2020A   
F                           PER-AVER                  PER-AVER   
Units                            TAF                       TAF   
1921-10-31                     972.0                    1067.0   
1921-11-30                     972.0                    1067.0   
1921-12-31                     972.0                    1067.0   
1922-01-31                     972.0                    1067.0   
1922-02-28                     972.0                    1067.0   

A                                                                             \
B          S_TRNTYLEVEL4DV_s0001 S_FOLSMLEVEL5DV_s0002 S_MELONLEVEL4DV_s0002   
C                  STORAGE-LEVEL         STORAGE-LEVEL         STORAGE-LEVEL   
D                           1MON                  1MON                  1MON   
E                         L2020A                L2020A                L2020A   
F                       PER-AVER              PER-AVER              PER-AVER   
Units                        TAF                   TAF                   TAF   
1921-10-31                1850.0                 712.0                1970.0   
1921-11-30                1850.0                 567.0                1970.0   
1921-12-31                1850.0                 567.0                1970.0   
1922-01-31                1900.0                 567.0                1970.0   
1922-02-28                2000.0                 567.0                1970.0   

A           ...                                                  \
B           ... S_SLUIS_SWPLEVEL4DV_s0062 S_TRNTYLEVEL4DV_s0062   
C           ...             STORAGE-LEVEL         STORAGE-LEVEL   
D           ...                      1MON                  1MON   
E           ...                    L2020A                L2020A   
F           ...                  PER-AVER              PER-AVER   
Units       ...                       TAF                   TAF   
1921-10-31  ...                    1067.0                1850.0   
1921-11-30  ...                    1067.0                1850.0   
1921-12-3

In [12]:
capacity_df = create_subset_list(var_df, variables_capacity)
capacity_df.head(5)

A                         CALSIM                            \
B          S_SHSTALEVEL6DV_s0001 S_SLUIS_CVPLEVEL5DV_s0001   
C                  STORAGE-LEVEL             STORAGE-LEVEL   
D                           1MON                      1MON   
E                         L2020A                    L2020A   
F                       PER-AVER                  PER-AVER   
Units                        TAF                       TAF   
1921-10-31                4552.0                     972.0   
1921-11-30                4552.0                     972.0   
1921-12-31                4552.0                     972.0   
1922-01-31                4552.0                     972.0   
1922-02-28                4552.0                     972.0   

A                                                           \
B          S_SLUIS_SWPLEVEL5DV_s0001 S_TRNTYLEVEL5DV_s0001   
C                      STORAGE-LEVEL         STORAGE-LEVEL   
D                               1MON                  1MON   
E                             L2020A                L2020A   
F                           PER-AVER              PER-AVER   
Units                            TAF                   TAF   
1921-10-31                    1067.0               2447.65   
1921-11-30                    1067.0               2447.65   
1921-12-31                    1067.0               2447.65   
1922-01-31                    1067.0               2447.65   
1922-02-28                    1067.0               2447.65   

A                     MANUAL-ADD                                               \
B          S_FOLSMLEVEL6DV_s0001 S_MLRTNLEVEL5DV_s0001 S_OROVLLEVEL6DV _s0001   
C                   STORAGE-ZONE          STORAGE-ZONE           STORAGE-ZONE   
D                           1MON                  1MON                   1MON   
E                         L2020A                L2020A                 L2020A   
F                        PER-CUM               PER-CUM                PER-CUM   
Units                        TAF                   TAF                    TAF   
1921-10-31                   967                   524                 3424.8   
1921-11-30                   967                   524                 3424.8   
1921-12-31                   967                   524                 3424.8   
1922-01-31                   967                   524                 3424.8   
1922-02-28                   967                   524                 3424.8   

A                                                CALSIM  \
B          S_MELONLEVEL5DV _s0001 S_SHSTALEVEL6DV_s0002   
C                    STORAGE-ZONE         STORAGE-LEVEL   
D                            1MON                  1MON   
E                          L2020A                L2020A   
F                         PER-CUM              PER-AVER   
Units                         TAF                   TAF   
1921-10-31                   2420                4552.0   
1921-11-30                   2420                4552.0   
1921-12-31                   2420                4552.0   
1922-01-31                   2420                4552.0   
1922-02-28                   2420                4552.0   

A                                     ...             MANUAL-ADD  \
B          S_SLUIS_CVPLEVEL5DV_s0002  ... S_OROVLLEVEL6DV _s0062   
C                      STORAGE-LEVEL  ...           STORAGE-ZONE   
D                               1MON  ...                   1MON   
E                             L2020A  ...                 L2020A   
F                           PER-AVER  ...                PER-CUM   
Units                            TAF  ...                    TAF   
1921-10-31                     972.0  ...                 3424.8   
1921-11-30                     972.0  ...                 3424.8   
1921-12-31                     972.0  ...                 3424.8   
1922-01-31                     972.0  ...                 3424.8   
1922-02-28                     972.0  ...                 3424.8   

A          

In [13]:
Apr_Storage_df = compute_annual_means_list(storage_df, variables_storage, units = units, months = [4])
# Apr_Storage_df.to_csv("Apr_Storage_df.csv")
Apr_Storage_df.head(5)

A                CALSIM                                            \
B         S_SHSTA_s0001 S_SHSTA_s0002 S_SHSTA_s0003 S_SHSTA_s0004   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          4552.1000     4552.1000     4552.1000     4552.1000   
1923          4263.1230     4239.9090     4171.1353     4010.7420   
1924          2834.6250     2546.8657     2359.4510     2438.9860   
1925          3846.4575     3557.0237     3232.7852     3108.3752   
1926          3872.9970     3687.7880     3290.8270     2963.1448   

A                                                                  \
B         S_SHSTA_s0005 S_SHSTA_s0006 S_SHSTA_s0007 S_SHSTA_s0008   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          4552.1000     4366.9140     4366.9140     4366.9140   
1923          3992.2300     4097.5460     4074.9663     4068.6558   
1924          2439.0420     2785.0469     2761.5180     2755.7869   
1925          2945.7876     3908.2537     3887.1030     3880.6790   
1926          2621.2085     4112.0140     4029.1260     4028.1350   

A                                      ...                              \
B         S_SHSTA_s0009 S_SHSTA_s0010  ... S_MELON_s0040 S_MELON_s0041   
C               STORAGE       STORAGE  ...       STORAGE       STORAGE   
D                  1MON          1MON  ...          1MON          1MON   
E                L2020A        L2020A  ...        L2020A        L2020A   
F              PER-AVER      PER-AVER  ...      PER-AVER      PER-AVER   
Units               TAF           TAF  ...           TAF           TAF   
WaterYear                              ...                               
1922          4366.9140     4366.9140  ...     1616.0999     1616.0999   
1923          4068.6145     4067.6423  ...     1855.5743     1855.5743   
1924          2751.9968     2739.6897  ...     1760.9857     1760.9857   
1925          3876.3730     3862.2240  ...     1640.2467     1640.2500   
1926          4010.6917     3999.5007  ...     1672.3463     1672.4313   

A                                                                  \
B         S_MELON_s0042 S_MELON_s0044 S_MELON_s0046 S_MELON_s0047   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          1616.0999     1664.6013     1459.0868     1727.1992   
1923          1855.5743     2028.7041     1563.0497     2120.1830   
1924          1760.9857     1834.6132     1279.2343     1772.7507   
1925          1640.2468     1858.9928     1062.6874     1906.6106   
1926          1672.4033     1828.6499      843.5307     1819.5507   

A                                                                  
B         S_MELON_s0051 S_MELON_s0056 S_MELON_s0062 S_MELON_s0063  
C               STORAGE       STORAGE       STORAGE       STORAGE  
D                  1MON          1MON          1MON          1MON  
E                L2020A        L2020A        L2020A        L2020A  
F  

In [14]:
Apr_Storage_CV_df = compute_cv_scenario_variable(Apr_Storage_df)
Apr_Storage_CV_df.head(5)

Variable,S_FOLSM,S_MELON,S_MLRTN,S_OROVL,S_SHSTA,S_SLUIS_CVP,S_SLUIS_SWP,S_TRNTY
Scenario,,,,,,,,
s0001,0.179558,0.228007,0.310850,0.212192,0.164362,0.278326,0.376141,0.215741
s0002,0.198544,0.252658,0.296839,0.230273,0.196397,0.316346,0.354917,0.239156
s0003,0.257935,0.288925,0.249659,0.289735,0.241658,0.395529,0.396077,0.275506
s0004,0.287553,0.321842,0.251359,0.326807,0.267786,0.430809,0.414987,0.298745
s0005,0.297249,0.347511,0.261610,0.348947,0.281576,0.457439,0.424623,0.307092


In [15]:
Sep_Storage_df = compute_annual_means_list(storage_df, variables_storage, units = units, months = [9])
# Sep_Storage_df.to_csv("Sep_Storage_df.csv")
Sep_Storage_df.head(5)

A                CALSIM                                            \
B         S_SHSTA_s0001 S_SHSTA_s0002 S_SHSTA_s0003 S_SHSTA_s0004   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          3399.9153    3376.82280     3020.8516     3073.8252   
1923          3220.0823    3014.23020     2927.9705     2918.5564   
1924          1226.1888     829.77466      550.0000      550.0000   
1925          2787.5560    2752.85940     2299.2434     2121.0837   
1926          2639.3853    2287.05700     1897.2922     1570.3761   

A                                                                  \
B         S_SHSTA_s0005 S_SHSTA_s0006 S_SHSTA_s0007 S_SHSTA_s0008   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          3186.2944     3400.0000     3400.0000     3400.0000   
1923          2938.1794     3032.7610     3010.6968     3100.0889   
1924           550.0000     1275.7760     1259.8429     1253.3451   
1925          1930.2318     3094.5890     3111.0310     3110.3342   
1926          1149.3823     2561.4998     2547.8848     2546.1170   

A                                      ...                              \
B         S_SHSTA_s0009 S_SHSTA_s0010  ... S_MELON_s0040 S_MELON_s0041   
C               STORAGE       STORAGE  ...       STORAGE       STORAGE   
D                  1MON          1MON  ...          1MON          1MON   
E                L2020A        L2020A  ...        L2020A        L2020A   
F              PER-AVER      PER-AVER  ...      PER-AVER      PER-AVER   
Units               TAF           TAF  ...           TAF           TAF   
WaterYear                              ...                               
1922          3400.0000     3400.0000  ...     1576.7302     1576.7302   
1923          3032.8640     2955.0242  ...     1740.3274     1740.3274   
1924          1249.0951     1234.8113  ...     1430.9294     1430.9327   
1925          3086.0720     3072.4958  ...     1536.7555     1536.8408   
1926          2506.1190     2500.0000  ...     1313.8268     1313.9225   

A                                                                  \
B         S_MELON_s0042 S_MELON_s0044 S_MELON_s0046 S_MELON_s0047   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          1576.7302     1775.9508    1443.45670     1714.2803   
1923          1740.3274     1861.4042    1322.61670     1818.9337   
1924          1430.9294     1452.2062     860.75305     1400.1252   
1925          1536.8127     1697.5591     797.99347     1686.7180   
1926          1313.8993     1431.1669     378.89517     1370.0459   

A                                                                  
B         S_MELON_s0051 S_MELON_s0056 S_MELON_s0062 S_MELON_s0063  
C               STORAGE       STORAGE       STORAGE       STORAGE  
D                  1MON          1MON          1MON          1MON  
E                L2020A        L2020A        L2020A        L2020A  
F  

In [16]:
Sep_Storage_CV_df = compute_cv_scenario_variable(Sep_Storage_df)
Sep_Storage_CV_df.head(5)

Variable,S_FOLSM,S_MELON,S_MLRTN,S_OROVL,S_SHSTA,S_SLUIS_CVP,S_SLUIS_SWP,S_TRNTY
Scenario,,,,,,,,
s0001,0.258190,0.275559,0.281327,0.356666,0.239686,0.864778,0.621431,0.299024
s0002,0.313035,0.305549,0.279120,0.403519,0.289071,0.922751,0.640207,0.329655
s0003,0.405338,0.357833,0.197802,0.442940,0.371235,1.033085,0.603005,0.366579
s0004,0.425055,0.394182,0.181338,0.485067,0.406971,1.099368,0.635779,0.395690
s0005,0.433445,0.425432,0.172041,0.510607,0.421263,1.053311,0.589250,0.401068


In [17]:
Apr_Capacity_df = compute_annual_means_list(capacity_df, variables_capacity, units = units, months = [4])
Apr_Capacity_df = strip_leveldv_from_columns(Apr_Capacity_df, level_name="B")
Apr_Capacity_df = strip_level_from_columns(Apr_Capacity_df, level_name="B")
# Apr_Capacity_df.to_csv("Apr_Capacity_df.csv")
Apr_Capacity_df.head(5)

A                CALSIM                                            \
B         S_SHSTA_s0001 S_SHSTA_s0002 S_SHSTA_s0003 S_SHSTA_s0004   
C         STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922             4552.0        4552.0        4552.0        4552.0   
1923             4552.0        4552.0        4552.0        4552.0   
1924             4552.0        4552.0        4552.0        4552.0   
1925             4552.0        4552.0        4552.0        4552.0   
1926             4552.0        4552.0        4552.0        4552.0   

A                                                                  \
B         S_SHSTA_s0005 S_SHSTA_s0006 S_SHSTA_s0007 S_SHSTA_s0008   
C         STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922             4552.0        4552.0        4552.0        4552.0   
1923             4552.0        4552.0        4552.0        4552.0   
1924             4552.0        4552.0        4552.0        4552.0   
1925             4552.0        4552.0        4552.0        4552.0   
1926             4552.0        4552.0        4552.0        4552.0   

A                                      ...    MANUAL-ADD                \
B         S_SHSTA_s0009 S_SHSTA_s0010  ... S_MELON_s0040 S_MELON_s0041   
C         STORAGE-LEVEL STORAGE-LEVEL  ...  STORAGE-ZONE  STORAGE-ZONE   
D                  1MON          1MON  ...          1MON          1MON   
E                L2020A        L2020A  ...        L2020A        L2020A   
F              PER-AVER      PER-AVER  ...       PER-CUM       PER-CUM   
Units               TAF           TAF  ...           TAF           TAF   
WaterYear                              ...                               
1922             4552.0        4552.0  ...        2420.0        2420.0   
1923             4552.0        4552.0  ...        2420.0        2420.0   
1924             4552.0        4552.0  ...        2420.0        2420.0   
1925             4552.0        4552.0  ...        2420.0        2420.0   
1926             4552.0        4552.0  ...        2420.0        2420.0   

A                                                                  \
B         S_MELON_s0042 S_MELON_s0044 S_MELON_s0046 S_MELON_s0047   
C          STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PER-CUM       PER-CUM       PER-CUM       PER-CUM   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922             2420.0        2420.0        2420.0        2420.0   
1923             2420.0        2420.0        2420.0        2420.0   
1924             2420.0        2420.0        2420.0        2420.0   
1925             2420.0        2420.0        2420.0        2420.0   
1926             2420.0        2420.0        2420.0        2420.0   

A                                                                  
B         S_MELON_s0051 S_MELON_s0056 S_MELON_s0062 S_MELON_s0063  
C          STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE  
D                  1MON          1MON          1MON          1MON  
E                L2020A        L2020A        L2020A        L2020A  
F  

In [18]:
Sep_Capacity_df = compute_annual_means_list(capacity_df, variables_capacity, units = units, months = [4])
Sep_Capacity_df = strip_leveldv_from_columns(Sep_Capacity_df, level_name="B")
Sep_Capacity_df = strip_level_from_columns(Sep_Capacity_df, level_name="B")
# Sep_Capacity_df.to_csv("Sep_Capacity_df.csv")
Sep_Capacity_df.head(5)

A                CALSIM                                            \
B         S_SHSTA_s0001 S_SHSTA_s0002 S_SHSTA_s0003 S_SHSTA_s0004   
C         STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922             4552.0        4552.0        4552.0        4552.0   
1923             4552.0        4552.0        4552.0        4552.0   
1924             4552.0        4552.0        4552.0        4552.0   
1925             4552.0        4552.0        4552.0        4552.0   
1926             4552.0        4552.0        4552.0        4552.0   

A                                                                  \
B         S_SHSTA_s0005 S_SHSTA_s0006 S_SHSTA_s0007 S_SHSTA_s0008   
C         STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F              PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922             4552.0        4552.0        4552.0        4552.0   
1923             4552.0        4552.0        4552.0        4552.0   
1924             4552.0        4552.0        4552.0        4552.0   
1925             4552.0        4552.0        4552.0        4552.0   
1926             4552.0        4552.0        4552.0        4552.0   

A                                      ...    MANUAL-ADD                \
B         S_SHSTA_s0009 S_SHSTA_s0010  ... S_MELON_s0040 S_MELON_s0041   
C         STORAGE-LEVEL STORAGE-LEVEL  ...  STORAGE-ZONE  STORAGE-ZONE   
D                  1MON          1MON  ...          1MON          1MON   
E                L2020A        L2020A  ...        L2020A        L2020A   
F              PER-AVER      PER-AVER  ...       PER-CUM       PER-CUM   
Units               TAF           TAF  ...           TAF           TAF   
WaterYear                              ...                               
1922             4552.0        4552.0  ...        2420.0        2420.0   
1923             4552.0        4552.0  ...        2420.0        2420.0   
1924             4552.0        4552.0  ...        2420.0        2420.0   
1925             4552.0        4552.0  ...        2420.0        2420.0   
1926             4552.0        4552.0  ...        2420.0        2420.0   

A                                                                  \
B         S_MELON_s0042 S_MELON_s0044 S_MELON_s0046 S_MELON_s0047   
C          STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PER-CUM       PER-CUM       PER-CUM       PER-CUM   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922             2420.0        2420.0        2420.0        2420.0   
1923             2420.0        2420.0        2420.0        2420.0   
1924             2420.0        2420.0        2420.0        2420.0   
1925             2420.0        2420.0        2420.0        2420.0   
1926             2420.0        2420.0        2420.0        2420.0   

A                                                                  
B         S_MELON_s0051 S_MELON_s0056 S_MELON_s0062 S_MELON_s0063  
C          STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE  STORAGE-ZONE  
D                  1MON          1MON          1MON          1MON  
E                L2020A        L2020A        L2020A        L2020A  
F  

In [19]:
Apr_StoPct_df = compute_percent_of_capacity(Apr_Storage_df, Apr_Capacity_df)
# Apr_StoPct_df.to_csv("Apr_StoPct_df.csv")
Apr_StoPct_df.head(5)

A                CALSIM                                            \
B         S_FOLSM_s0001 S_FOLSM_s0002 S_FOLSM_s0003 S_FOLSM_s0004   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PERCENT       PERCENT       PERCENT       PERCENT   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          93.071355     93.071355     93.071355     93.071355   
1923          93.071355     93.071355     93.071355     93.071525   
1924          41.554984     41.644391     32.770757     37.074323   
1925          93.071355     93.071355     93.071355     80.918649   
1926          90.052534     86.170917     77.502172     80.917177   

A                                                                  \
B         S_FOLSM_s0005 S_FOLSM_s0006 S_FOLSM_s0007 S_FOLSM_s0008   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PERCENT       PERCENT       PERCENT       PERCENT   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          93.071355     93.071355     93.071355     93.071355   
1923          93.071835     93.071355     93.071355     93.071355   
1924          33.993912     38.711359     41.444503     40.913024   
1925          74.192658     93.071355     93.071355     93.071355   
1926          81.006996     87.496019     89.987952     90.058339   

A                                      ...                              \
B         S_FOLSM_s0009 S_FOLSM_s0010  ... S_TRNTY_s0040 S_TRNTY_s0041   
C               STORAGE       STORAGE  ...       STORAGE       STORAGE   
D                  1MON          1MON  ...          1MON          1MON   
E                L2020A        L2020A  ...        L2020A        L2020A   
F               PERCENT       PERCENT  ...       PERCENT       PERCENT   
Units               TAF           TAF  ...           TAF           TAF   
WaterYear                              ...                               
1922          93.071355     93.071355  ...     77.405683     77.405683   
1923          93.071355     93.071355  ...     79.851208     79.851208   
1924          40.306982     44.510645  ...     53.022940     53.022904   
1925          93.071355     93.071355  ...     66.349560     66.349556   
1926          87.162343     86.590936  ...     75.955578     76.008498   

A                                                                  \
B         S_TRNTY_s0042 S_TRNTY_s0044 S_TRNTY_s0046 S_TRNTY_s0047   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PERCENT       PERCENT       PERCENT       PERCENT   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          77.406418     77.565265     64.576823     83.880898   
1923          79.831406     72.820857     49.119678     70.044676   
1924          54.640292     46.556358     33.532999     35.291474   
1925          66.398260     67.309366     43.385182     63.052765   
1926          76.034703     72.898302     41.678265     62.434641   

A                                                                  
B         S_TRNTY_s0051 S_TRNTY_s0056 S_TRNTY_s0062 S_TRNTY_s0063  
C               STORAGE       STORAGE       STORAGE       STORAGE  
D                  1MON          1MON          1MON          1MON  
E                L2020A        L2020A        L2020A        L2020A  
F  

In [20]:
Sep_StoPct_df = compute_percent_of_capacity(Sep_Storage_df, Sep_Capacity_df)
# Sep_StoPct_df.to_csv("Sep_StoPct_df.csv")
Sep_StoPct_df.head(5)

A                CALSIM                                            \
B         S_FOLSM_s0001 S_FOLSM_s0002 S_FOLSM_s0003 S_FOLSM_s0004   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PERCENT       PERCENT       PERCENT       PERCENT   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          73.778459     73.965253     73.911799     73.903092   
1923          62.186381     63.242740     58.291324     65.972982   
1924          28.112947     26.763553      9.307135      9.307135   
1925          66.829835     67.708873     60.331954     60.663915   
1926          47.422754     44.686719     48.056737     48.830832   

A                                                                  \
B         S_FOLSM_s0005 S_FOLSM_s0006 S_FOLSM_s0007 S_FOLSM_s0008   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PERCENT       PERCENT       PERCENT       PERCENT   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          73.780465     77.598046     73.244264     77.596629   
1923          64.428718     61.283526     67.744340     67.744436   
1924           9.307135     28.831768     31.225024     30.950915   
1925          61.122341     57.003213     59.528325     59.745806   
1926          45.233340     41.952195     41.639690     41.609506   

A                                      ...                              \
B         S_FOLSM_s0009 S_FOLSM_s0010  ... S_TRNTY_s0040 S_TRNTY_s0041   
C               STORAGE       STORAGE  ...       STORAGE       STORAGE   
D                  1MON          1MON  ...          1MON          1MON   
E                L2020A        L2020A  ...        L2020A        L2020A   
F               PERCENT       PERCENT  ...       PERCENT       PERCENT   
Units               TAF           TAF  ...           TAF           TAF   
WaterYear                              ...                               
1922          77.512995     71.261489  ...     77.925643     77.925618   
1923          67.743888     67.743837  ...     63.118044     63.114252   
1924          30.367580     34.084512  ...     28.871418     28.871415   
1925          56.665491     56.086077  ...     61.137393     61.145527   
1926          40.814058     40.136460  ...     46.691565     46.519114   

A                                                                  \
B         S_TRNTY_s0042 S_TRNTY_s0044 S_TRNTY_s0046 S_TRNTY_s0047   
C               STORAGE       STORAGE       STORAGE       STORAGE   
D                  1MON          1MON          1MON          1MON   
E                L2020A        L2020A        L2020A        L2020A   
F               PERCENT       PERCENT       PERCENT       PERCENT   
Units               TAF           TAF           TAF           TAF   
WaterYear                                                           
1922          77.927036     66.155962     48.589198     61.542373   
1923          62.856160     51.376806     40.251580     40.033783   
1924          28.924348     29.947182     20.930059     18.262949   
1925          61.186624     58.760562     37.086673     47.459285   
1926          46.357343     44.365698     26.286606     37.092932   

A                                                                  
B         S_TRNTY_s0051 S_TRNTY_s0056 S_TRNTY_s0062 S_TRNTY_s0063  
C               STORAGE       STORAGE       STORAGE       STORAGE  
D                  1MON          1MON          1MON          1MON  
E                L2020A        L2020A        L2020A        L2020A  
F  

In [21]:
Apr_StoPct_CV_df = compute_cv_scenario_variable(Apr_StoPct_df)
Apr_StoPct_CV_df.head(5)

Variable,S_FOLSM,S_MELON,S_MLRTN,S_OROVL,S_SHSTA,S_SLUIS_CVP,S_SLUIS_SWP,S_TRNTY
Scenario,,,,,,,,
s0001,0.179558,0.228007,0.310850,0.212192,0.164362,0.278326,0.376141,0.215741
s0002,0.198544,0.252658,0.296839,0.230273,0.196397,0.316346,0.354917,0.239156
s0003,0.257935,0.288925,0.249659,0.289735,0.241658,0.395529,0.396077,0.275506
s0004,0.287553,0.321842,0.251359,0.326807,0.267786,0.430809,0.414987,0.298745
s0005,0.297249,0.347511,0.261610,0.348947,0.281576,0.457439,0.424623,0.307092


In [22]:
Sep_StoPct_CV_df = compute_cv_scenario_variable(Sep_StoPct_df)
Sep_StoPct_CV_df.head(5)

Variable,S_FOLSM,S_MELON,S_MLRTN,S_OROVL,S_SHSTA,S_SLUIS_CVP,S_SLUIS_SWP,S_TRNTY
Scenario,,,,,,,,
s0001,0.258190,0.275559,0.281327,0.356666,0.239686,0.864778,0.621431,0.299024
s0002,0.313035,0.305549,0.279120,0.403519,0.289071,0.922751,0.640207,0.329655
s0003,0.405338,0.357833,0.197802,0.442940,0.371235,1.033085,0.603005,0.366579
s0004,0.425055,0.394182,0.181338,0.485067,0.406971,1.099368,0.635779,0.395690
s0005,0.433445,0.425432,0.172041,0.510607,0.421263,1.053311,0.589250,0.401068


In [23]:
# Apr_Flood_df = compute_annual_means_list(flood_df, variables_floodpool, units = units, months = [4])
# Apr_Flood_df = strip_leveldv_from_columns(Apr_Flood_df, level_name="B")
# Apr_Flood_df = strip_level_from_columns(Apr_Flood_df, level_name="B")
# # Apr_Flood_df.to_csv("Apr_Flood_df.csv")
# Apr_Flood_df.head(5)

In [24]:
# Sep_Flood_df = compute_annual_means_list(flood_df, variables_floodpool, units = units, months = [9])
# Sep_Flood_df = strip_leveldv_from_columns(Sep_Flood_df, level_name="B")
# Sep_Flood_df = strip_level_from_columns(Sep_Flood_df, level_name="B")
# # Sep_Flood_df.to_csv("Sep_Flood_df.csv")
# Sep_Flood_df.head(5)

In [25]:
# Apr_SpillFreq_df, Apr_SpillProb_df = freq_and_prob_storage_ge_flood(Apr_Storage_df, Apr_Flood_df)
# Apr_SpillFreq_df.head(10)

In [26]:
# Sep_SpillFreq_df, Sep_SpillProb_df = freq_and_prob_storage_ge_flood(Sep_Storage_df, Sep_Flood_df)
# Sep_SpillFreq_df.head(10)

In [27]:
flood_clean_df = strip_leveldv_from_columns(flood_df, level_name="B")
flood_clean_df = strip_level_from_columns(flood_clean_df, level_name="B")

In [28]:
flood_clean_df.head(10)

A                 CALSIM                                            \
B          S_FOLSM_s0001 S_MELON_s0001 S_MLRTN_s0001 S_OROVL_s0001   
C          STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL STORAGE-LEVEL   
D                   1MON          1MON          1MON          1MON   
E                 L2020A        L2020A        L2020A        L2020A   
F               PER-AVER      PER-AVER      PER-AVER      PER-AVER   
Units                TAF           TAF           TAF           TAF   
1921-10-31         712.0     1970.0000         439.0        3057.0   
1921-11-30         567.0     1970.0000         439.0        3057.0   
1921-12-31         567.0     1970.0000         439.0        2828.8   
1922-01-31         567.0     1970.0000         439.0        3000.0   
1922-02-28         567.0     1970.0000         439.0        2712.6   
1922-03-31         756.0     2030.0000         478.5        2819.9   
1922-04-30         900.0     2220.0000         524.0        3334.8   
1922-05-31         967.0     2315.0437         524.0        3425.2   
1922-06-30         967.0     2420.0000         524.0        3425.2   
1922-07-31         942.0     2420.0000         524.0        3425.2   

A                                                                           \
B          S_SHSTA_s0001 S_SLUIS_CVP_s0001 S_SLUIS_SWP_s0001 S_TRNTY_s0001   
C          STORAGE-LEVEL     STORAGE-LEVEL     STORAGE-LEVEL STORAGE-LEVEL   
D                   1MON              1MON              1MON          1MON   
E                 L2020A            L2020A            L2020A        L2020A   
F               PER-AVER          PER-AVER          PER-AVER      PER-AVER   
Units                TAF               TAF               TAF           TAF   
1921-10-31      3250.000             972.0            1067.0        1850.0   
1921-11-30      3252.100             972.0            1067.0        1850.0   
1921-12-31      3359.160             972.0            1067.0        1850.0   
1922-01-31      3833.280             972.0            1067.0        1900.0   
1922-02-28      4139.735             972.0            1067.0        2000.0   
1922-03-31      4483.766             972.0            1067.0        2100.0   
1922-04-30      4552.100             972.0            1067.0        2300.0   
1922-05-31      4552.100             972.0            1067.0        2420.0   
1922-06-30      4500.000             972.0            1067.0        2447.0   
1922-07-31      4150.000             972.0            1067.0        2270.0   

A                                       ...                                  \
B          S_FOLSM_s0002 S_MELON_s0002  ... S_SLUIS_SWP_s0062 S_TRNTY_s0062   
C          STORAGE-LEVEL STORAGE-LEVEL  ...     STORAGE-LEVEL STORAGE-LEVEL   
D                   1MON          1MON  ...              1MON          1MON   
E                 L2020A        L2020A  ...            L2020A        L2020A   
F               PER-AVER      PER-AVER  ...          PER-AVER      PER-AVER   
Units                TAF           TAF  ...               TAF           TAF   
1921-10-31         712.0     1970.0000  ...            1067.0        1850.0   
1921-11-30         567.0     1970.0000  ...            1067.0        1850.0   
1921-12-31         567.0     1970.0000  ...            1067.0        1850.0   
1922-01-31         567.0     1970.0000  ...            1067.0        1900.0   
1922-02-28         567.0     1970.0000  ...            1067.0        2000.0   
1922-03-31         756.0     2030.0000  ...            1067.0        2100.0   
1922-04-30         900.0     2220.0000  ...            1067.0        2300.0   
1922-05-31         967.0     2314.9304  ...            1067.0        2420.0   
1922-06-30         967.0     2420.0000  ...            1067.0        2447.0   
1922-07-31         942.0     2420.0000  ...            1067.0        2270.0   

A                                                                   \
B          S_FOLSM_s0063 S_MELON_s0063 S_MLRTN_s0063 S_OROVL_s0063  

In [29]:
SpillFreq_df, SpillProb_df = freq_and_prob_storage_ge_flood(storage_df, flood_clean_df)
SpillFreq_df.head(10)

variable,S_FOLSM,S_MELON,S_MLRTN,S_OROVL,S_SHSTA,S_SLUIS_CVP,S_SLUIS_SWP,S_TRNTY
scenario,,,,,,,,
s0001,518,179,62,258,400,94,79,219
s0002,508,196,60,252,378,92,88,209
s0003,407,145,79,199,267,66,72,133
s0004,366,133,76,173,231,56,62,111
s0005,336,120,79,160,211,50,51,97
s0006,<NA>,197,143,244,364,102,69,208
s0007,<NA>,197,143,242,362,105,192,215
s0008,<NA>,197,143,240,361,106,216,217
s0009,<NA>,197,143,242,361,109,227,218


In [30]:
SpillProb_df.head(10)

variable,S_FOLSM,S_MELON,S_MLRTN,S_OROVL,S_SHSTA,S_SLUIS_CVP,S_SLUIS_SWP,S_TRNTY
scenario,,,,,,,,
s0001,0.035972,0.012431,0.004306,0.017917,0.027778,0.006528,0.005486,0.015208
s0002,0.035278,0.013611,0.004167,0.017500,0.026250,0.006389,0.006111,0.014514
s0003,0.028264,0.010069,0.005486,0.013819,0.018542,0.004583,0.005000,0.009236
s0004,0.025417,0.009236,0.005278,0.012014,0.016042,0.003889,0.004306,0.007708
s0005,0.023333,0.008333,0.005486,0.011111,0.014653,0.003472,0.003542,0.006736
s0006,NaN,0.013681,0.009931,0.016944,0.025278,0.007083,0.004792,0.014444
s0007,NaN,0.013681,0.009931,0.016806,0.025139,0.007292,0.013333,0.014931
s0008,NaN,0.013681,0.009931,0.016667,0.025069,0.007361,0.015000,0.015069
s0009,NaN,0.013681,0.009931,0.016806,0.025069,0.007569,0.015764,0.015139


## Save outputs

In [31]:
Apr_Storage_df.to_csv(Apr_Storage_Volume_path)
Sep_Storage_df.to_csv(Sep_Storage_Volume_path)
Apr_Storage_CV_df.to_csv(Apr_Storage_Volume_CV_path)
Sep_Storage_CV_df.to_csv(Sep_Storage_Volume_CV_path)
Apr_StoPct_df.to_csv(Apr_Storage_Percent_path)
Sep_StoPct_df.to_csv(Sep_Storage_Percent_path)
Apr_StoPct_CV_df.to_csv(Apr_Storage_Percent_CV_path)
Sep_StoPct_CV_df.to_csv(Sep_Storage_Percent_CV_path)
SpillFreq_df.to_csv(Spill_Frequency_path)
SpillProb_df.to_csv(Spill_Probability_path)

In [32]:
print("Done!")

Done!
